# Подготовка данных для обучения моделей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* https://scikit-learn.org/stable/modules/compose.html#pipeline-chaining-estimators
* https://pytorch.org/docs/stable/data.html 
* https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
* https://codingnomads.com/pytorch-dataset-to-dataloader-using-collate-fn
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann


In [134]:
import pandas as pd
import torch as th
from torch.utils.data import Dataset
import numpy as np
from typing import List,Union, Callable, Any

th.device("cuda" if th.cuda.is_available() else "cpu")

pd.options.mode.copy_on_write = True

## Задачи для совместного разбора

1. Создайте синтетический датасет для задачи регрессии и представьте его в виде `torch.utils.data.Dataset`

In [135]:
from torch.utils.data import Dataset
from sklearn.datasets import make_regression
import torch

class RegressionDataset(Dataset):
    
    def __init__(self, transform = None):
        X,y = make_regression()
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)
        self.transform = transform
        
    def __getitem__(self, index):
        if self.transform is not None:
            return self.transform(self.X[index]), self.y[index]
        return self.X[index], self.y[index]
    
    def __len__(self):
        return len(self.X)

class MyTransform:
    def __init__(self, multiplier):
        self.multiplier = multiplier
    
    def __call__(self, X):
        return X * self.multiplier

In [136]:
dataset = RegressionDataset()
dataset[0]

(tensor([ 1.5059e+00, -5.9457e-01, -2.1079e-01,  3.2462e-01, -1.6466e+00,
          8.9474e-02,  4.3163e-01, -6.9158e-01, -8.7991e-01,  2.3244e-01,
          2.1269e+00,  4.6654e-01, -2.5385e-03,  1.2243e+00,  9.4604e-01,
          8.5614e-01,  1.3846e+00, -2.1194e+00,  1.3622e+00, -1.3566e+00,
         -3.8628e-01, -5.8444e-01,  8.0903e-01, -7.7575e-01, -5.6347e-02,
         -3.4480e-01,  5.3788e-01, -1.4797e+00, -9.1454e-01,  3.4794e-01,
         -1.2835e+00, -2.8357e-01,  6.8979e-02,  9.9300e-02, -3.0969e-01,
         -2.3645e-01, -2.4474e-02, -1.0653e+00, -7.8687e-01, -7.4883e-01,
          7.5742e-01, -1.4171e+00, -2.4942e-02, -4.6717e-01,  2.1206e-01,
          3.4641e-01, -9.0924e-01,  4.6325e-01, -6.5014e-01,  1.2516e-01,
          1.0779e+00,  5.1047e-01,  1.9822e+00,  7.1337e-01,  1.0854e+00,
         -1.8131e+00,  4.9319e-01,  3.7006e-01, -6.5457e-02,  1.3430e-01,
         -5.3937e-01,  1.2341e+00,  1.0368e-01, -5.7141e-02,  1.1539e+00,
          9.2126e-01, -2.5190e-01,  9.

In [137]:
# %load_ext cudf.pandas
# import pandas as pd

In [138]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=16, shuffle=True, drop_last=True)

for X,y in loader:
    print(X.shape, y.shape)

torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])
torch.Size([16, 100]) torch.Size([16])


## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файл `bank-full.csv` ([источник](https://www.kaggle.com/datasets/hariharanpavan/bank-marketing-dataset-analysis-classification)) в виде `pd.DataFrame`.

Опишите класс `BankDatasetBase`. Решение должно удовлетворять следующим критериям:

* класс наследуется от `torch.utils.data.Dataset`;
* при создании объекта в конструктор передается набор данных в виде `pd.DataFrame`;
* объекты класса имеют поля `X` и `y` с признаками и метками соответственно;
* класс реализует интерфейс последовательностей (`__getitem__` + `__len__`);
* `obj[i]` возвращает кортеж, содержащий `i`-ую строку из `obj.X` (серию) и `i`-ую строку из `obj.y` (строку).
    
Создайте объект класса `BankDatasetBase` и продемонстрируйте работоспособность.

- [ ] Проверено на семинаре

In [139]:
class BankDatasetBase(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        if data.empty:
            raise ValueError("DataFrame не должен быть пустым.")

        self.X = data.iloc[:, :-1]
        self.y = data.iloc[:, -1]       
        

    def __getitem__(self, idx: int) -> tuple:
        x = self.X.loc[idx]
        y = self.y.loc[idx]


        return (x, y)

    def __len__(self) -> int:
        return len(self.X)
    
bank_full = pd.read_csv("bank-full.csv")
display(bank_full)


dataset = BankDatasetBase(bank_full)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [140]:
len(dataset)

45211

In [141]:
dataset[0]

(age                  58
 job          management
 marital         married
 education      tertiary
 default              no
 balance            2143
 housing             yes
 loan                 no
 contact         unknown
 day                   5
 month               may
 duration            261
 campaign              1
 pdays                -1
 previous              0
 poutcome        unknown
 Name: 0, dtype: object,
 'no')

<p class="task" id="2"></p>

2\. Опишите класс `BankDataset`. Решение должно удовлетворять всем критериям из предыдущего задания, а также:
* при создании объекта в конструктор может быть передан необязательные аргументы `transform` и `target_transform`;
* если аргумент `transform` был передан, то при получении `i`-го элемента, нужно вызвать `transform(x)` и вернуть полученный результат.
* если аргумент `target_transform` был передан, то при получении `i`-го элемента, нужно вызвать `target_transform(y)` и вернуть полученный результат.

Создайте объект класса `BankDataset` и продемонстрируйте работоспособность (без передачи `target_transform` и `transform`).

- [ ] Проверено на семинаре

In [142]:
class BankDataset(BankDatasetBase):
    def __init__(
        self, 
        data: pd.DataFrame, 
        transform: Callable | None = None,
        target_transform: Callable | None = None
    ) -> None:
        super().__init__(data)
        
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, idx: int) -> tuple[th.Tensor, Any]:
        x,y = super().__getitem__(idx)
        if self.transform is not None:
            x = self.transform(x)
        
        if self.target_transform is not None:
            y = self.target_transform(y)
        
        
        
        
        
        
        
        return x, y

In [143]:
bank_full = pd.read_csv("bank-full.csv")
# display(bank_full)
    
data = BankDataset(bank_full, lambda x: x*22, lambda y: y*3)

data[10]

(age                                                        902
 job          admin.admin.admin.admin.admin.admin.admin.admi...
 marital      divorceddivorceddivorceddivorceddivorceddivorc...
 education    secondarysecondarysecondarysecondarysecondarys...
 default           nononononononononononononononononononononono
 balance                                                   5940
 housing      yesyesyesyesyesyesyesyesyesyesyesyesyesyesyesy...
 loan              nononononononononononononononononononononono
 contact      unknownunknownunknownunknownunknownunknownunkn...
 day                                                        110
 month        maymaymaymaymaymaymaymaymaymaymaymaymaymaymaym...
 duration                                                  4884
 campaign                                                    22
 pdays                                                      -22
 previous                                                     0
 poutcome     unknownunknownunknownunkno

<p class="task" id="3"></p>

3\. Опишите класс `OrdinalEncoderTransform`. Решение должно удовлетворять следующим критериям:

* при создании объекта в конструктор передаются названия нечисловых столбцов в датасете
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (признаки) и возвращает набор признаков, в котором нечисловые характеристики закодированы целыми числами;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объект класса `OrdinalEncoderTransform`.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию. 

- [ ] Проверено на семинаре

In [144]:
class OrdinalEncoderTransform:
    def __init__(self, categorical_feature_names:List):
        self.categorical_feature_names = categorical_feature_names
        self.dicts = dict()


    def __call__(self, x):
        all_names_list = x.columns if len(x.shape)==2 else x.index
        if set(self.categorical_feature_names) & set(all_names_list) == set():
            raise KeyError(f'There are no columns {self.categorical_feature_names} in x')
        
        for name in self.categorical_feature_names:
                if name in all_names_list:
                    if isinstance(x[name],str):
                        if name in self.dicts.keys():
                            if x[name] in self.dicts[name].keys():
                                x[name] = self.dicts[name][x[name]]
                            else:
                                the_val = max(list(self.dicts[name].values()))+1
                                self.dicts[name][x[name]] = the_val
                                x[name] = the_val
                        else:
                            new_dict = dict()
                            new_dict[x[name]] = 0
                            self.dicts[name] = new_dict
                            x[name] = 0
                    else:
                        if name in self.dicts.keys():
                            existing_set = set(self.dicts[name].keys())
                            not_existing_set = set(x[name].values) 
                            if not_existing_set & existing_set == not_existing_set:
                                x[name] = x[name].map(self.dicts[name])
                            else:
                                if not_existing_set & existing_set == set():
                                    for i in x[name].values:
                                        self.dicts[name][i] = max(list(self.dicts[name].values()))+1 if list(self.dicts[name].values())!= [] else 1
                                    x[name] = x[name].map(self.dicts[name])
                                else:
                                    not_existing_set.update(existing_set)
                                    for i in list(not_existing_set.difference(existing_set)):
                                        self.dicts[name][i] = max(list(self.dicts[name].values()))+1
                                    x[name] = x[name].map(self.dicts[name])
                        else:
                            self.dicts[name] = dict(list(zip(x[name].unique(),range(len(x[name].unique())))))
                            x[name] = x[name].map(self.dicts[name])
                        
                else:
                    raise KeyError(f'There is no column {name} in x')
                
                
        return x
    
ordinal_job = OrdinalEncoderTransform(['job'])
data = BankDataset(bank_full,transform=ordinal_job)

In [145]:
# работает и с единичными случаями
data[534]

(age                 52
 job                  0
 marital        married
 education    secondary
 default             no
 balance           1236
 housing            yes
 loan                no
 contact        unknown
 day                  6
 month              may
 duration           247
 campaign             1
 pdays               -1
 previous             0
 poutcome       unknown
 Name: 534, dtype: object,
 'no')

In [146]:
# работает и с множественными случаями
data[:534]

(     age  job  marital  education default  balance housing loan  contact  day  \
 0     58    3  married   tertiary      no     2143     yes   no  unknown    5   
 1     44    9   single  secondary      no       29     yes   no  unknown    5   
 2     33    5  married  secondary      no        2     yes  yes  unknown    5   
 3     47   10  married    unknown      no     1506     yes   no  unknown    5   
 4     33    8   single    unknown      no        1      no   no  unknown    5   
 ..   ...  ...      ...        ...     ...      ...     ...  ...      ...  ...   
 530   44   10  married    primary      no      213      no   no  unknown    6   
 531   31    1  married    primary      no      203     yes   no  unknown    6   
 532   42    7   single  secondary      no      518     yes   no  unknown    6   
 533   40    3   single   tertiary      no     3877     yes   no  unknown    6   
 534   52    0  married  secondary      no     1236     yes   no  unknown    6   
 
     month  du

In [147]:
ordinal_job.dicts

{'job': {'admin.': 0,
  'housemaid': 1,
  'unemployed': 2,
  'management': 3,
  'retired': 4,
  'entrepreneur': 5,
  'student': 6,
  'services': 7,
  'unknown': 8,
  'technician': 9,
  'blue-collar': 10,
  'self-employed': 11}}

<p class="task" id="4"></p>

4\. Опишите класс `LabelEncoderTransform`. Решение должно удовлетворять следующим критериям:

* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` имеет один параметр (строку) и возвращает целое число, соответствующее этой строке;
* состояние объекта (индексы для кодирования) обновляется в момент очередного вызова `__call__` (т.е. все данные сразу никогда не передаются никакому методу объекта).
    
Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании объекта в качестве аргумента `target_transform` объект класса `LabelEncoderTransform`.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию. 

- [ ] Проверено на семинаре

In [148]:
class LabelEncoderTransform:
    def __init__(self):
        self.perevodict = dict()


    def __call__(self, y):
        if isinstance(y,str):
            if y in self.perevodict.keys():
                return self.perevodict[y]
            else:
                self.perevodict[y] = max(list(self.perevodict.values()))+1 if list(self.perevodict.values())!= [] else 0
                return self.perevodict[y]
        else:
            existing_set = set(self.perevodict.keys())
            not_existing_set = set(y.values)
            if not_existing_set & existing_set == not_existing_set:
                y = y.map(self.perevodict)
            else:
                if not_existing_set & existing_set == set():
                    for i in not_existing_set:
                        self.perevodict[i] = max(list(self.perevodict.values()))+1 if list(self.perevodict.values())!= [] else 0
                    y = y.map(self.perevodict)
                else:
                    not_existing_set.update(existing_set)
                    for i in list(not_existing_set.difference(existing_set)):
                        self.perevodict[i] = max(list(self.perevodict.values()))+1 if list(self.perevodict.values())!= [] else 0
                    y = y.map(self.perevodict)
            return y
                    

labelenc = LabelEncoderTransform()
data = BankDataset(bank_full,target_transform=labelenc)

In [149]:
# Работает как в разовых случаях

data[534]

(age                 52
 job             admin.
 marital        married
 education    secondary
 default             no
 balance           1236
 housing            yes
 loan                no
 contact        unknown
 day                  6
 month              may
 duration           247
 campaign             1
 pdays               -1
 previous             0
 poutcome       unknown
 Name: 534, dtype: object,
 0)

In [150]:
data[45207]

(age                71
 job           retired
 marital      divorced
 education     primary
 default            no
 balance          1729
 housing            no
 loan               no
 contact      cellular
 day                17
 month             nov
 duration          456
 campaign            2
 pdays              -1
 previous            0
 poutcome      unknown
 Name: 45207, dtype: object,
 1)

In [151]:
# так и во множественных
data[45206:]

(       age           job   marital  education default  balance housing loan  \
 45206   51    technician   married   tertiary      no      825      no   no   
 45207   71       retired  divorced    primary      no     1729      no   no   
 45208   72       retired   married  secondary      no     5715      no   no   
 45209   57   blue-collar   married  secondary      no      668      no   no   
 45210   37  entrepreneur   married  secondary      no     2971      no   no   
 
          contact  day month  duration  campaign  pdays  previous poutcome  
 45206   cellular   17   nov       977         3     -1         0  unknown  
 45207   cellular   17   nov       456         2     -1         0  unknown  
 45208   cellular   17   nov      1127         5    184         3  success  
 45209  telephone   17   nov       508         4     -1         0  unknown  
 45210   cellular   17   nov       361         2    188        11    other  ,
 45206    1
 45207    1
 45208    1
 45209    0
 45210 

<p class="task" id="5"></p>

5\. Опишите класс `ToTensor`.  Решение должно удовлетворять следующим критериям:
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает на вход серию или фрейм и возвращает тензор.

Опишите класс `Compose`.  Решение должно удовлетворять следующим критериям:
* при создании объекта в конструктор передается список объектов `transforms`, каждый из которых имеет метод `__call__(x, y)`;
* класс реализует интерфейс `Callable` (`__call__`); метод `__call__` принимает имеет параметра (признаки и класс в числовом виде) и и возвращает кортеж, полученный путем последовательного вызова объектов из `transforms`.

Продемонстрируйте работоспособность, создав объект `BankDataset` и передав при создании преобразования `Compose` список из объектов LabelEncoderTransform и ToTensor.

Важно: не создавайте копию класса `BankDataset` с добавленными в него возможностями этого преобразования, используйте композицию. 

- [ ] Проверено на семинаре

In [ ]:
class ToTensor:
    def __init__(self):
        pass

    def __call__(self, x: pd.Series|pd.DataFrame):
        if isinstance(x, (pd.Series, pd.DataFrame)):       
            cat_cols_idxs = []
            for obj in x.loc[*[x.index[0] for i in range(len(x.shape)-1)]]:
                if type(obj)==type('str'):
                    cat_cols_idxs.append(False)
                else:
                    cat_cols_idxs.append(True)
                    
            sumx = sum(cat_cols_idxs)
                    
            cat_cols_idxs = th.tensor(cat_cols_idxs)
            mask = cat_cols_idxs.repeat(*x.shape[:-1],1).numpy()
            xape = list(x.shape)
            xape[-1] = sumx
            
            resx = x.values[mask].reshape(xape)
            resx = np.float64(resx)
            return th.tensor(resx)
        else:
            return th.tensor(x)
            # raise ValueError(f'You gave neither pd.Series nor pd.DataFrame. Object of type {type(x)} cannot be converted')
    
class Compose:
    def __init__(self, transforms: List[Callable]):
        self.transforms = transforms
    
    def __call__(self, x):
        for obj in self.transforms:
            x = obj(x)
        return x
        
comp = Compose([LabelEncoderTransform(), ToTensor()])
data = BankDataset(bank_full,target_transform=comp)

data[:3]

(   age           job  marital  education default  balance housing loan  \
 0   58    management  married   tertiary      no     2143     yes   no   
 1   44    technician   single  secondary      no       29     yes   no   
 2   33  entrepreneur  married  secondary      no        2     yes  yes   
 3   47   blue-collar  married    unknown      no     1506     yes   no   
 
    contact  day month  duration  campaign  pdays  previous poutcome  
 0  unknown    5   may       261         1     -1         0  unknown  
 1  unknown    5   may       151         1     -1         0  unknown  
 2  unknown    5   may        76         1     -1         0  unknown  
 3  unknown    5   may        92         1     -1         0  unknown  ,
 tensor([0., 0., 0., 0.], dtype=torch.float64))

<p class="task" id="6"></p>

6\. Разделите датасет из предыдущего задания на обучающую и тестовую выборку в соотношении 75% на 25%. Создайте объект `DataLoader` для получения пакетов размера 64, полученных из перемешанного обучающего датасета. Кастомизируйте `DataLoader` (используйте аргумент `collate_fn`) таким образом, чтобы пакет признаков был представлен в виде трехмерного тензора размера 64x2x8 (разделите 16 признаков на два тензора по 8). Получите один пакет и выведите на экран размерность тензоров пакета. 



- [ ] Проверено на семинаре

In [153]:
compx = Compose([OrdinalEncoderTransform(bank_full.iloc[:, :-1].select_dtypes(include=['object']).columns), ToTensor()])
compy = Compose([LabelEncoderTransform(), ToTensor()])
dataset = BankDataset(bank_full, transform=compx, target_transform=compy)

def collate_fn(batch):
    # 64 строки в батче
    # признак выдается в виде трехмерного тензора размера 64x2x8
    return th.stack([i[0] for i in batch]).view(-1, 2, 8), th.stack([i[1] for i in batch])

loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True, collate_fn=collate_fn)

for X,y in loader:
    print(X.shape, y.shape)
    break

torch.Size([64, 2, 8]) torch.Size([64])
